<a href="https://colab.research.google.com/github/Bulbatronik/Helbling_STARTHACK25/blob/main/notebooks/MemoryBuffer_to_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# %%capture
!pip install langchain>=0.1.17 openai>=1.13.3 langchain_openai>=0.1.6 transformers>=4.40.1 langchain_community pypdf -q

In [57]:
import os, io, requests, pprint, json
from pypdf import PdfReader

from openai import OpenAI

from langchain.chains.conversation.memory import ConversationSummaryMemory
from langchain import OpenAI, LLMChain#, PromptTemplate
from langchain.memory import ConversationSummaryMemory
from langchain.prompts import PromptTemplate

from google.colab import userdata

# Loading an LLM

In [58]:
# Extract menu from PDF
menu_url = "https://restaurantcandela.ch/wp-content/uploads/2025/02/Mittagsmenu-14.pdf"  # Replace with your PDF URL

try:
    response = requests.get(menu_url)
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    pdf_file = io.BytesIO(response.content)
    pdf_reader = PdfReader(pdf_file)

    menu_text = ""
    for page in pdf_reader.pages:
        menu_text += page.extract_text() + "\n"

    # Clean up the menu text (remove extra whitespace, etc.)
    menu_text = " ".join(menu_text.split())

    print(menu_text)
except requests.exceptions.RequestException as e:
    menu_text = f"Error fetching menu: {e}"
except Exception as e:
    menu_text = f"Error processing PDF: {e}"


Gerne geben wir Ihnen detailliert über die Herkunft unserer Gerichte Auskunft Alle Preise inkl. 8.1 % MwSt. Mittags Empfehlungen Tagessalat 7.50 Tagessuppe – Pastinaken-Lauchcremesuppe mit Schnittlauchöl 7.50 *** Fisch Teller 25.50 Duo vom Lachs * und Zander ** auf Linsen-Dal an Curry-Kokossauce dazu Mango-Espuma, Kokoschips und Koriander Fisch Empfehlung 35.50 Gebratenes Zanderfilet ** an Kräutersauce Zitronen-Kartoffelpüree, Blattspinat & Gartenkresseöl Fleisch Teller 24.50 Paccheri-Pasta «al Ragoût» mit Bolognesesauce dazu Parmesanspäne, knuspriger Salbei und Tomatenöl Küchenchef-Empfehlung 42.00 Kalbsfilet-Schnitzel an Zitronensauce Pommes Frites und Blattspinat Rind-Empfehlung 43.00 Rindsfiletwürfel an rassiger «Stroganoffsauce» Quarkspätzli & Blattspinat Rinds-Hackbraten an Rotweinsauce 32.00 Quarkspätzli & Marktgemüse Burger Empfehlung 28.50 Hausburger im Spezial-Maisbun mit Käse, Adobo-Chili-Mayonnaise, knusprige Zwiebeln, Avocadocreme, Eisbergsalat dazu Pommes Frites Vegetaris

## Memory

In [61]:
# Assume you had these identifiers
session_id = "client_id_123"
chat_session_id = "chat_session_id_456"

In [67]:
llm_main = OpenAI(api_key=userdata.get("OPENAI_KEY"))
llm_summarizer = OpenAI(api_key=userdata.get("OPENAI_KEY"))

# Template for the summurizing LLM (memory)
template_summary = """Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

Current summary:
{summary}

New lines of the conversation:
{new_lines}

New updated memories:
"""

summary_prompt = PromptTemplate(
    input_variables=["new_lines", "summary"],
    template=template_summary)


# Define the type of memory we will use
memory = ConversationSummaryMemory(
    llm=llm_summarizer,
    memory_key="memories",
    prompt=summary_prompt
)


LOAD_MEMORY = True
# Try to load past memories (if available)
if LOAD_MEMORY and os.path.exists(f"{session_id}-{chat_session_id}-output.json"):
    with open(f"{session_id}-{chat_session_id}-output.json", "r") as file:
        memories = json.load(file)
    memory.buffer = memories["memories"]

print(memory.load_memory_variables({}))

{'memories': "Seva introduces themselves as a customer and expresses their desire to make an order. Alex, the waiter, greets Seva and offers to show them the menu or take their order if they already know what they would like. Seva asks for a recommendation and Alex suggests the special dish of salmon and zander on a bed of lentil dal with a curry coconut sauce, served with mango espuma, coconut chips, and cilantro. Alex mentions that it's a customer favorite and asks if Seva would like to try it. Seva then asks about the dish of the day, and Alex enthusiastically describes the salmon and zander dish again, emphasizing that it's highly recommended. Seva decides to order the dish of the day and confirms their choice with Alex. Alex thanks Seva for their order and confirms the details of the dish. Seva thanks Alex and wishes them a nice day."}


## LLM

In [68]:
######################### LLM (MAIN)
template_main = f"""You are a waiter named Alex at a restaurant called Aarau, serving clients. Your responsibilities include:
  - Providing or showing the menu
  - Taking food and drink orders from all guests at a table
  - Delivering orders to clients
  - Clearing dishes and handling payments after meals
  - Guiding clients to locations like the bar, table, or restroom
  - Engaging in small talk with clients

    Menu:
    {menu_text}

    You have access to memories summarizing past interactions with the client. You can use them (IF NOT EMPTY) to recomment something based on the past preferences of the user:
    Memories:
    {{memories}}

    Respond based on the above information, the user's input, and any memories. Be patient and allow the user time to think. After taking an order, always repeat it back for confirmation.
    When the user finishes ordering, thank them and let them know their order is on its way.
    Stay in character as Alex and do not role-play as other characters. You also need to understand when the conversation is ongoing and when it is a new meeting not to greet the user many times.

    Human:
    {{input_prompt}}

    AI:
"""

prompt = PromptTemplate(
    template=template_main,
    input_variables=["input_prompt", "memories"]
)


# Chain the LLM, prompt, and memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm_main,
    memory=memory
)

### Run interaction


In [69]:
pprint.pp(llm_chain.invoke({"input_prompt": "Hi! My name is Seva, I would like to make an order!"}))
pprint.pp(llm_chain.invoke({"input_prompt": "What can you recommend me?"}))
pprint.pp(llm_chain.invoke({"input_prompt": "What's the dish of the day?"}))
pprint.pp(llm_chain.invoke({"input_prompt": "How much does it cost?"}))
pprint.pp(llm_chain.invoke({"input_prompt": "Do you have hot ice on the menu?"}))
pprint.pp(llm_chain.invoke({"input_prompt": "Okay, I'll take the dish of the day. Thank you and have a nice day!"}))

{'input_prompt': 'Hi! My name is Seva, I would like to make an order!',
 'memories': 'Seva introduces themselves as a customer and expresses their '
             'desire to make an order. Alex, the waiter, greets Seva and '
             'offers to show them the menu or take their order if they already '
             'know what they would like. Seva asks for a recommendation and '
             'Alex suggests the special dish of salmon and zander on a bed of '
             'lentil dal with a curry coconut sauce, served with mango espuma, '
             "coconut chips, and cilantro. Alex mentions that it's a customer "
             'favorite and asks if Seva would like to try it. Seva then asks '
             'about the dish of the day, and Alex enthusiastically describes '
             "the salmon and zander dish again, emphasizing that it's highly "
             'recommended. Seva decides to order the dish of the day and '
             'confirms their choice with Alex. Alex thanks Seva 

In [70]:
memory.load_memory_variables({})

{'memories': "Current summary:\nHuman: Yes, I'll have the dish of the day.\nAI:     Great! I'll place the order for you. Thank you for choosing our restaurant.\nHuman: How much does it cost?\nAI:     The prices for our dishes vary. The Tagessalat is 7.50, the Tagessuppe is 7.50, and the Fisch Teller is 25.50. The Mittags Empfehlungen Tagessalat is 7.50, the Tagessuppe is 7.50, and the Fisch Empfehlung is 35.50. The Mittags Empfehlungen Tagessalat is 7.50, the Tagessuppe is 7.50, and the Fisch Teller is 25.50. The Mittags Empfehlungen Tagessalat is 7.50, the Tagessuppe is 7.50, and the Fisch Empfehlung is 35.50. The Mittags Empfehlungen Tagessalat is 7.50, the Tagessuppe is 7.50, and the Fisch Teller is 25.50.\n\nHuman: Okay, I'll take the dish"}

In [71]:
# End the session -> store the memories in the respective json
output = memory.load_memory_variables({})
# Save to a json file
with open(f"{session_id}-{chat_session_id}-output.json", "w") as file:
    file.write(json.dumps(output, indent=4))